In [1]:
# install boto3 

# !pip install boto3 --upgrade
# pip install aws-utils
# pip install aws-python-utils

In [30]:
# curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg
# wget https://aws-dlc-sample-models.s3.amazonaws.com/pytorch/imagenet_classes.txt

In [31]:
import torch, torchvision
import PIL
from torchvision import transforms
from PIL import Image

def get_image(filename):
    im = Image.open(filename)
    # ImageNet pretrained models required input images to have width/height of 224
    # and color channels normalized according to ImageNet distribution.
    im_process = transforms.Compose([transforms.Resize([224, 224]),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])])
    im = im_process(im) # 3 x 224 x 224
    return im.unsqueeze(0) # Add dimension to become 1 x 3 x 224 x 224

im = get_image('/home/jay/Downloads/sudhanshu.jpeg')

# eval() toggles inference mode
model = torchvision.models.resnet50(pretrained=True).eval()
# Compile model to TorchScript via tracing
# Here want to use the first attached accelerator, so we specify ordinal 0.
with torch.jit.optimized_execution(True):
  # You can trace with any input
  model = torch.jit.trace(model, im)

# Serialize model
torch.jit.save(model, 'resnet50_traced.pt')

# Deserialize model
model = torch.jit.load('resnet50_traced.pt', map_location=torch.device('cpu'))

# Perform inference. Make sure to disable autograd and use EI execution context
with torch.no_grad():
    with torch.jit.optimized_execution(True):
        probs = model(im)

# Torchvision implementation doesn't have Softmax as last layer.
# Use Softmax to convert activations to range 0-1 (probabilities)
probs = torch.nn.Softmax(dim=1)(probs)

# Get top 5 predicted classes
classes = eval(open('/home/jay/imagenet_classes.txt').read())
pred_probs, pred_indices = torch.topk(probs, 5)
pred_probs = pred_probs.squeeze().numpy()
pred_indices = pred_indices.squeeze().numpy()

for i in range(len(pred_indices)):
    curr_class = classes[pred_indices[i]]
    curr_prob = pred_probs[i]
    print('{}: {:.4f}'.format(curr_class, curr_prob))

jersey, T-shirt, tee shirt: 0.0760
beer glass: 0.0347
bottlecap: 0.0240
cellular telephone, cellular phone, cellphone, cell, mobile phone: 0.0224
drumstick: 0.0201
